# Argentic RAG 
Dieser Prototyp untersucht die Fähigkeiten von LLM unterstützen Multiagent System für eine Sequenzanalyse 

Als Framework wird crewai verwendet. 


In [ ]:
""" %pip install crewai
%pip install load_dotenv
%pip install os
%pip install crewai[tools]
%pip install agentops
%pip install langchain_openai """

Setup der Fachfragen an die Transkripte

In [ ]:
Fall = "F5"
Text = "I1: Moment erstmal. (9) Ich komm mir so herzlos vor, wenn ich jetzt auf die Struktur zurückführe. Aber trotzdem, wollen Sie dazu überhaupt eine Antwort haben? #02:32:14-7#"

Initialisierung der Dokumente

In [ ]:
from crewai import Agent, Task, Crew, Process 
import os
from crewai_tools import DirectorySearchTool, PDFSearchTool, SerperDevTool
from langchain_openai import ChatOpenAI

os.environ["OPENAI_MODEL_NAME"]="gpt-4o"

doctool = DirectorySearchTool(directory=f'anon\\F5')

import agentops
agentops.init()# Update pip



## Starten der Untersuchung


In [ ]:
import time
timestamp = time.time() 

Sequenzer = Agent(
    role = 'Sequenzer',
    goal = "Dein Ziel ist die Aufteilung eines gegebenen Textes in Sequenzen, hierbei sollen Satzzeichen, Pausen (3) und Sinngrenzen beachtung finden."
           "Keine Beachtung finden die Sprecher markierung und der Zeitstempel am Ende.",
    backstory = 'Du bist Sequenzierungsprofi und kennst die Techniken der Objektiven Hermeneutik und kannst diese Anwenden',
    verbose = True,
    allow_delegation= False,
    
)

Storyteller = Agent(
    role = 'Geschichtenerzähler',
    goal = "Ziel ist es, mindestens vier vielfältige und phantasievolle Geschichten für jede Sequenz zu schaffen."
            "Diese Geschichten sollten eine Reihe von Kontexten abdecken, um eine breite Basis für die Interpretation zu schaffen.",
    backstory = 'Du bist eine Kreativer Geschichten Schreiber ',
    verbose = True,
    allow_delegation= False,
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.9)
   
)

Interpreter = Agent(
    role = 'Interpreter',
    goal = 'Ziel gemeinsame Themen oder Interpretationen ',
    backstory = 'Der Interpreter analysiert die vom Geschichtenerzähler geschaffenen Geschichten und leitet daraus gemeinsame Themen oder Interpretationen ab.'
                'Dies beinhaltet die Identifizierung von zugrunde liegenden Mustern und Bedeutungen, die in den verschiedenen Geschichten übereinstimmen.',
    verbose = True,
    allow_delegation= False,
    )

Contextualizer = Agent(
    role = 'Kontextualizer',
    goal = 'Ziel ist ein vergleich der Interpretationen mit dem realen Kontext.',
    backstory = 'Der Kontextualisierer vergleicht die vom Interpreten abgeleiteten Interpretationen mit einem bestimmten realen Kontext.'
                'Dabei wird bewertet, wie gut die Interpretationen mit der tatsächlichen Situation übereinstimmen, und es werden Erkenntnisse über die Relevanz und Genauigkeit der Interpretationen gewonnen.',
    verbose = True,
    allow_delegation= False,
    tools = [doctool]
    )


sequence_task = Task(
    description=f'Sequenziere den Text: \" {Text} \" in Sinnabschnitte, wobei auch die Pausen, markiert durch Klammern z.B. (5) Beachtung finden sollen ',
    expected_output='Eine Liste mit Sequenzen. ', 
    agent=Sequenzer,
    output_file= f"Ergebnisse\\{Fall}\\{timestamp}_{Fall}_aRAG_Seq_Sequenz.txt",
    )
    

storytelling_task  = Task( 
    description=f'Erstelle vier kreative Geschichten zu jeder Sequenz!',
    expected_output='Eine Liste von Geschichten, die verschiedene Kontexte und Szenarien für jede Sequenz', 
    agent=Storyteller,
    Context=sequence_task, 
  # tools=[doctool],
    output_file= f"Ergebnisse\\{Fall}\\{timestamp}_{Fall}_aRAG_Seq_Storytelling.txt")

interpretation_task = Task( 
    description= 'Analysiere die vom Geschichtenerzähler geschaffenen Geschichten und leitet daraus gemeinsame Themen oder Interpretationen ab!',
    expected_output=f'Ziel gemeinsame Themen oder Interpretationen', 
    agent=Interpreter,
   # tools=[doctool], 
    output_file= f"Ergebnisse\\{Fall}\\{timestamp}_{Fall}_aRAG_Seq_Interpreter.txt")

context_confrontation_task = Task( 
    description= 'Vergleiche die Interpretationen mit einem bestimmten realen Kontext.'
                 'Bewerte wie gut die Interpretationen mit der tatsächlichen Situation übereinstimmen, und ermittle die Relevanz und Genauigkeit der Interpretationen.',
    expected_output=f' Ein detaillierter Vergleich der Interpretationen mit dem realen Kontext, der die Übereinstimmung und eventuelle Abweichungen hervorhebt', 
    agent=Contextualizer,
    tools=[doctool], 
    output_file= f"Ergebnisse\\{Fall}\\{timestamp}_{Fall}_aRAG_Seq_Confrontation.txt")

In [ ]:
# Form the crew with a sequential process
hier_report_crew = Crew(
  agents=[Sequenzer, Storyteller, Interpreter, Contextualizer],
  tasks=[sequence_task, storytelling_task, interpretation_task,context_confrontation_task],
  process=Process.hierarchical,
  manager_llm=ChatOpenAI(temperature=0, model="gpt-4o"),
  verbose=2,
  memory=False,
  cache=True,
  share_crew=False,
  full_output=True,
  output_log_file=True,
  language = "de",
  language_File = "de.json"

)

Starten der Untersuchung

In [ ]:
import time
#result = report_crew.kickoff(inputs={"directory": f'anon\\{Fall}'})
result = hier_report_crew.kickoff(inputs={'Text':Text , 'timestamp': timestamp })
#result_seq = seq_report_crew.kickoff(Text)
print("######################")
print(result)
print("######################")

In [ ]:
print(hier_report_crew.usage_metrics)